In [ ]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime
import os

In [ ]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [ ]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str}, parse_dates=['period_date'])

In [ ]:
pp_dep.period_date.unique()

In [ ]:
pp_dep.indicateur.unique()

In [ ]:
corres = {
    "Nombre de bâtiments de l’Etat retenus pour faire l’objet d’une rénovation énergétique" : 
        ['Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE1'],
    "Nombre de bonus écologiques et de primes à la conversion versés " : 
        ['Nombre de primes à la conversion - DVP1',
        'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE1'],
    "Nombre de projets industriels soutenus" : [ 'Nombre d’entreprises - BPI2',
                                                'Nombre d’entreprises bénéficiaires - PIT2', 
                                                'Nombre d’entreprises bénéficiaires - SAC2'],
    "Nombre de chèques export et chèques VIE (volontariat international en entreprise" : 
        ['Nombre d’entreprises bénéficiares - SBF1'],
    "Nombre de contrats d’apprentissage ayant bénéficié de l’aide exceptionnelle " : 
        ['Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP1'],
    "Nombre d’aides versées pour l’embauche des jeunes" : 
        ["Nombre d'aides à l'embauche des jeunes - PEJ1"],
    "Nombre de jeunes entrés en CIE (contrats initiative emploi) et PEC (parcours emploi compétences) " : 
        ['Entrées de jeunes en CIE - CIE1', 'Entrées de jeunes en PEC - PEC1'], 
    "Renforcement des subventions de Business France (chèque export, chèque VIE)" : 
        ['Nombre d’entreprises bénéficiaires - SBF1']
}

In [ ]:
pd.DataFrame.from_dict(corres, orient='index').to_csv("correspondances.csv")

In [ ]:
# On récupère la liste des indicateurs à afficher
list_indicateurs = list(itertools.chain.from_iterable(corres.values()))
len(list_indicateurs)

In [ ]:
pp_dep_indics = pp_dep[pp_dep['indicateur'].isin(list_indicateurs)].reset_index(drop=True)
pp_dep_indics.drop('Unnamed: 0', axis=1, inplace=True)
pp_dep_indics.shape

In [ ]:
# Ne garder que la date la plus récente pour chaque indicateur
def get_latest_date_for_indic_and_dep(indicateur, dep):
    df = pp_dep_indics[(pp_dep_indics['indicateur'] == indicateur) & 
                          (pp_dep_indics['dep'] == dep)]
    return df['period_date'].max()


pp_dep_to_export = pp_dep_indics.loc[pp_dep_indics.apply(lambda x: get_latest_date_for_indic_and_dep(x['indicateur'], x['dep']) == x['period_date'], axis=1)]
pp_dep_to_export.reset_index(drop=True, inplace=True)
pp_dep_to_export

In [ ]:
export = pd.pivot_table(pp_dep_to_export, columns=['dep', 'reg', 'departement', 'region'],
                        index=['mesure', 'indicateur'], values="valeur")
#export.columns = export.columns.get_level_values(1)
#export['National'] = export.sum(axis=1)
export.reset_index(inplace=True)
export["index"] = export["mesure"] + "/" + export["indicateur"]
export.set_index('index', inplace=True)
export.drop(columns=["mesure", "indicateur"], inplace=True)
export = export.T
#export.fillna(0, inplace=True)
export = export.loc[~export.index.isin(["00"], level=1)]

In [ ]:
export["Nombre de bonus écologiques et de primes à la conversion versés"] = \
    export["Prime à la conversion des véhicules légers/Nombre de primes à la conversion - DVP1"] + \
    export["Bonus écologique/Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE1"]

export["Nombre de projets industriels soutenus"] = (
    export["Modernisation des filières automobiles et aéronautiques/Nombre d’entreprises - BPI2"] +
    export["Relocalisation : soutien aux projets industriels dans les territoires/Nombre d’entreprises bénéficiaires - PIT2"] +
    export["Relocalisation : sécurisation des approvisionnements critiques/Nombre d’entreprises bénéficiaires - SAC2"])

export["Nombre de jeunes entrés en CIE et PEC"] = (
    export["PEC jeunes/Entrées de jeunes en PEC - PEC1"] +
    export["CIE jeunes/Entrées de jeunes en CIE - CIE1"]

)
export

In [ ]:
mkdir_ifnotexist('exports')

### Rénovation bâtiments états

In [ ]:
tcsv = export["Rénovation bâtiments Etat/Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE1"]
tcsv.to_csv("exports/2_renovation_batiment_etat.csv", sep=";")

### Bonus écologique

In [ ]:
tcsv = export["Nombre de bonus écologiques et de primes à la conversion versés"]
tcsv.to_csv("exports/3_bonus_ecologique.csv", sep=";")

### Soutien industriel 

In [ ]:
tcsv = export["Nombre de projets industriels soutenus"]
tcsv.to_csv("exports/4_soutien_industriel.csv", sep=";")

### Soutien Business France

In [ ]:
tcsv = export["Renforcement des subventions de Business France (chèque export, chèque VIE)/Nombre d’entreprises bénéficiaires - SBF1"]
tcsv.to_csv("exports/5_cheque_export_vie_sbf.csv", sep=";")

### Apprentissage

In [ ]:
tcsv = export["Apprentissage/Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP1"]
tcsv.to_csv("exports/6_apprentissage.csv", sep=";")

### Emploi Jeune

In [ ]:
tcsv = export["Prime à l'embauche des jeunes/Nombre d'aides à l'embauche des jeunes - PEJ1"]
tcsv.to_csv("exports/7_embauche_jeunes.csv", sep=";")

### CIE/PEC

In [ ]:
tcsv = export["Nombre de jeunes entrés en CIE et PEC"]
tcsv.to_csv("exports/8_cie_pec.csv", sep=";")